## 自动驾驶控制算法

1. 《无人驾驶车辆控制模型预测控制》龚建伟(第二版)
2. 《车辆动力学及控制》

自动驾驶主要分为四大块：

1. 感知（基于视觉、激光、视觉和激光融合 + 高精地图）。
2. 决策
3. 路径规划
4. 控制 (PID、LQR、MPC)

自动驾驶还有一个难点，就是芯片的计算速度(台式机性能的芯片)，车载芯片工作环境极为恶劣。

## 三个坐标系与运动学方程

百度阿波罗采用的是PID和LQR算法，MPC效果比较好，但是计算耗时比较严重。控制算法的前提是路径规划。

控制主要分为两个部分：

1. **纵向控制**：油门/刹车 -> 力 -> 加速度 -> 速度 -> 位置
2. **横向控制**：前轮转角 -> 横向位移 （航向角）

横向控制难道不需要与纵向控制相互配合来达到横向位移吗？这里就需要引入三个坐标系

<img src="../images/18-zuobiaoxi.png" width="25%">

### 绝对坐标系、大地坐标系、（X, Y）



### 车身坐标系（x，y）



### 自然坐标系（z，n）（Frenet坐标系）

Frenet坐标系可以使得控制算法变得简单，并且可以和轨迹规划的坐标系对应上，轨迹规划就对应到Frenet坐标系。

Frenet坐标系还有一个优点，就是可以将纵向控制和横向控制解耦。

<img src="../images/18-youshouxi.png" width="45%">

在一些文章中，理解坐标系的变换是个难点，因为一些学物理的搞控制和学计算机的使用的坐标系可能会不一样，有的是使用右手坐标系有的是使用时左手坐标系。不同的坐标系可能会直接导致计算的过程中某些量的正负。

本文使用的是右手坐标系：

## 车辆建模

一般都会将车进行简化，将4个轮子变成两个轮子的自行车模型。也就是认为这个车是完全的左右对称的，并且左轮和右轮的轮距，相对于运动轨迹来说可以忽略不计。就将其压扁了，压成了一条线。这么做是为了简化问题，抓住主要矛盾。

<img src="../images/18-hangxiangjiao.png" width="25%">

假设这里有个自行车模型，后轮的速度是$v$，前轮的速度是$v^{\prime}$, 质心的速度是$v_{c}$。一般来说，车质心的速度方向和车轴向的方向不重合。

车的轴线方向和绝对坐标系X轴向方向一般也不重合，这样的话就定义两个角度，第一个角度是$\varphi$, 第二个角度是$\beta$。

<img src="../images/zixingchemoxing.png" width="65%">

1. 横摆角：表示车的轴线与大地坐标系（绝对坐标系）的X轴的夹角
2. 质心侧偏角：质心速度与车身坐标系的x轴的夹角
3. 航向角：质心速度与**大地坐标系（绝对坐标系)**的X轴的夹角

上图中的前轮转角和后轮转角都要带引号，因为是做了假设的。有三个假设：

1. 车左右对称。
2. 车的轮距忽略不计，可以压扁成一条线。
3. 车的轮子有无穷的刚度。因此轮子会变形，这个变形不能忽略。

前轮转角和方向盘转角不能一一对应的这个现象叫做**侧偏**。

除此之外，还有一些其他物理量：

<img src="../images/18-qita.png" width="65%">

对这个$v$沿着车身坐标系进行正交分解，沿着小$x$方向的，定义为纵向车速$v_{x}$，沿着小$y$方向的定义为横向车速$v_{y}$。同样可以得到纵向加速度$a_{x}$，和侧向加速度$a_{y}$。

之后我们就可以定义微分方程，这个微分方程中我们就可以得到前轮转角$\delta_{f}$是怎么影响侧向位移与航向角的。建立这个微分方程也分为两种：

1. 依据几何关系建立的, 运动学模型。

2. 依据牛顿力学建立的, 动力学模型。(会考虑轮胎的变形。复杂，参数难以给定)。

## 运动学模型

<img src="../images/18-donglixue.png" width="25%">

一个刚体的角速度，等于它的线速度到这个速度质心的距离。这样就得到了一个微分方程，但是这个微分方程没有考虑前轮转角。还需要进一步考虑：

设质心到前轮的距离为$a$，质心到后轮的距离为$b$。依据正弦定理，我们有：

$$
\begin{aligned}
&\frac{a}{\sin \left(\delta_{f}-\beta\right)}=\frac{R}{\sin \left(\frac{\pi}{2}-\delta_{f}\right)} \\
&\frac{b}{\sin (\delta_{r} + \beta)}=\frac{R}{\left.\sin \mid \frac{\pi}{2}-\delta_{r}\right)}
\end{aligned}
$$

$$
\frac{a}{R}=\frac{\sin \left(\delta_{f}-\beta\right)}{\sin \left(\frac{\pi}{2}-\delta_{f} \right)}=\frac{\sin \delta_{f} \cos \beta-\sin \beta \cos \delta_{f}}{\cos \delta_{f}}=\tan \delta_{f} \cos \beta-\sin \beta
$$

$$
\frac{b}{R}=\frac{\sin \left(\delta_{r}+\beta\right)}{\sin \left( \frac{\pi}{2}-\delta_{r} \right)}=\frac{\sin \delta_{r} \cos \beta+\sin \beta \cos \delta_{r} }{\cos \delta_{r}}=\tan \delta_{r} \cos \beta+\sin \beta
$$

$$
\frac{a+b}{R}=\left(\tan \delta_{f}+\tan \delta_{r}\right) \quad \frac{1}{R}=\frac{\tan \delta_{f}+\tan \delta_{r}}{L} \quad L=a+b \quad \text {为车的轴距}
$$

此时的运动学方程可以描述为：

$$
\begin{aligned}
&\dot{x}=V \cos (\varphi+\beta) \\
&\dot{y}=V \sin (\varphi+\beta) \\
&\dot{\varphi}=\frac{V}{R}=\frac{v\left(\tan \delta_{f}+\tan \delta_{r}\right)}{L}
\end{aligned}
$$

上述推倒是在低速条件下的，认为车不会发生侧向滑动，也就是不会发生漂移，$v_{y} \approx 0$。

$$
\beta=\arctan \frac{v_{y}}{v_{x}}=0
$$

一般后轮不转向，又是在低速条件下，所以近似$\delta_{r} \approx 0$。


最终的运动学方程可以描述为：

$$
\begin{aligned}
&\dot{x}=V \cos (\varphi) \\
&\dot{y}=V \sin (\varphi) \\
&\dot{\varphi}=\frac{V}{R}=\frac{v \tan \delta_{f} }{L}
\end{aligned}
$$

其中这个$\varphi$就是横摆角，$v$就是质心的速度。可以看出，就是通过速度和前轮摆角来控制位置和横摆角的。